In [1]:
from datetime import datetime, timedelta

In [2]:
import calendar

In [3]:
import requests

In [4]:
import grequests

In [5]:
import pandas as pd

In [6]:
from pandas import DataFrame, Series

In [7]:
import json

In [8]:
validParameters = ["starttime", "endtime", "updatetime", "minlatitude", "maxlatitude", "minlongitude", "maxlongitude"]

In [9]:
parameters = {
    "starttime2": "2015-1-1", 
    "endtime2": "2015-12-1", 
    "products": "focal-mechanism",
    "minlongitude": "-125.69510773828125",
    "maxlongitude": "-119.31754426171875",
    "minlatitude": "36.682994957749884",
    "maxlatitude": "38.79394764206436",
}

In [10]:
MAX_EVENTS = 10000
TIME_FORMAT = '%Y-%m-%dT%H:%M:%S'
DATE_FORMAT = '%Y-%m-%d'

In [11]:
def validDate(dt):
    if type(dt) is datetime:
        return true
    
    if type(dt) is str:
        try:
            parsed = datetime.strptime(dt,TIME_FORMAT)
        except:
            try:
                parsed = datetime.strptime(dt,DATE_FORMAT)
            except:
                return False
        return True

    if type(dt) is int:
        try:
            parsed = datetime.fromtimestamp(dt)
        except:
            return False

In [12]:
def prepareParameters(parameters):
    if parameters is None:
        parameters = {}
    elif type(parameters) is not dict:
        raise Exception("Expecting a dictionary of parameter values")
       
    if ("starttime" not in parameters) or (not validDate(parameters["starttime"])):
        parameters["starttime"] = datetime(2013, 1, 1).strftime(DATE_FORMAT)
    if ("endtime" not in parameters) or (not validDate(parameters["endtime"])):
        parameters["endtime"] = datetime.utcnow().strftime(DATE_FORMAT)
        
    return parameters

In [13]:
def splitDateRange(start, end, interval):
    if (not validDate(start)) and (not validDate(end)):
        raise Exception("Invalid date range: {0}-{1}".format(start, end))
    
    startdatetime = datetime.strptime(start, DATE_FORMAT)
    enddatetime = datetime.strptime(end, DATE_FORMAT)
    num_days = (enddatetime - startdatetime).days
    
    date_list = [(startdatetime + timedelta(days=x*interval), startdatetime + timedelta(days=(x+1)*interval)) 
                 for x in range(0, int(num_days / interval))]
    
    #append final date interval for remainder of days
    if date_list[-1][1] < enddatetime:
        date_list.append((date_list[-1][1], enddatetime))
        
    return date_list

In [14]:
def createQueries(params):
    # check the number of events returned by the query.  if over the max limit, split request into time segments
    # and return the list of requests as an array
    # if the request is under the max limit, return an array of 1, which will contain the original query
    queries = []
    
    params = prepareParameters(params)
    
    queryString = "&".join(["{0}={1}".format(key, params[key]) for key in params if key in validParameters])
    
    countUrl = "http://earthquake.usgs.gov/fdsnws/event/1/count?%s" % queryString
    print(countUrl)
    result = requests.get(countUrl)
    
    count = 0
    if result.status_code == 200:
        count = int(result.text)
    else:
        print("Error getting count: [{0}] {1}".format(result.status_code, result.text))
    
    queryUrl = "http://earthquake.usgs.gov/fdsnws/event/1/query.geojson?"
    
    if count > MAX_EVENTS:
        date_list = splitDateRange(params["starttime"], params["endtime"], 30)
        print("Need to break up queries. {0} results found.".format(count))
        for startdate, enddate in date_list:
            cpparams = params.copy()
            cpparams["starttime"] = startdate.strftime(DATE_FORMAT)
            cpparams["endtime"] = enddate.strftime(DATE_FORMAT)
            queryString = "&".join(["{0}={1}".format(key, cpparams[key]) for key in cpparams if key in validParameters])
            
            queries.append("{0}{1}".format(queryUrl, queryString))
    elif count == 0:
        print("No results match that query.")
    else:
        print("Single query is fine. {0} results found.".format(count))
        queries.append("{0}{1}".format(queryUrl, queryString))
        
    return queries

In [15]:
def processResponse(response, **data):
    if response.status_code != 200:
        print("ERROR processing response.  Status: {0} Text: {1}".format(response.status_code, response.text))
        return

    return response

In [16]:
def requestErrorHandler(response, exception):
    print("Request failed: {0}".format(exception))

In [17]:
def loadEventData(params):
    parameters = prepareParameters(params)
    summaryUrlList = createQueries(parameters)
    
    masterDataFrame = None
    
    #use first request to initialize DataFrame and then queue up remaining queries
    response = requests.get(summaryUrlList[0])
    if response.status_code == 200:
        json_data = response.json()

        features = DataFrame(json_data["features"])
        masterDataFrame = pd.concat([features, features["properties"].apply(pd.Series)], axis=1)

        # process the remaining requests by sending them asynchronously through grequests
        summaryUrlRequests = [grequests.get(url, hooks = {'response': processResponse }) 
                              for url in summaryUrlList[1:]]

        for resp in grequests.imap(summaryUrlRequests, exception_handler=requestErrorHandler):
            json_data = resp.json()

            features = DataFrame(json_data["features"])
            main = pd.concat([features, features["properties"].apply(pd.Series)], axis=1)

            masterDataFrame = masterDataFrame.append(main, ignore_index = True)
            print(masterDataFrame.shape)
            
    return masterDataFrame

In [18]:
eventData = loadEventData(parameters)

http://earthquake.usgs.gov/fdsnws/event/1/count?minlatitude=36.682994957749884&maxlatitude=38.79394764206436&maxlongitude=-119.31754426171875&starttime=2013-01-01&endtime=2016-03-31&minlongitude=-125.69510773828125
Need to break up queries. 17137 results found.
(525, 30)
(888, 30)
(1331, 30)
(1767, 30)
(2187, 30)
(2612, 30)
(3097, 30)
(3552, 30)
(4027, 30)
(4522, 30)
(4978, 30)
(5397, 30)
(5844, 30)
(6305, 30)
(6682, 30)
(7055, 30)
(7514, 30)
(7972, 30)
(8449, 30)
(9197, 30)
(9583, 30)
(10034, 30)
(10497, 30)
(10912, 30)
(11307, 30)
(11778, 30)
(12212, 30)
(12717, 30)
(13143, 30)
(13522, 30)
(13954, 30)
(14353, 30)
(14747, 30)
(15499, 30)
(15879, 30)
(16270, 30)
(16564, 30)
(16748, 30)
(17137, 30)


In [19]:
eventData

,geometry,id,properties,type,alert,cdi,code,detail,dmin,felt,...,sources,status,time,title,tsunami,type,types,tz,updated,url
0,"{'type': 'Point', 'coordinates': [-121.5548, 3...",nc71929426,"{'time': 1359251751100, 'tz': -480, 'type': 'e...",Feature,None,3.1,71929426,http://earthquake.usgs.gov/fdsnws/event/1/quer...,0.035933,10,...,",nc,us,ci,",REVIEWED,1359251751100,"M 2.8 - 2km SSW of San Juan Bautista, California",0,earthquake,",dyfi,general-link,geoserve,impact-text,nearby...",-480,1457699561770,http://earthquake.usgs.gov/earthquakes/eventpa...
1,"{'type': 'Point', 'coordinates': [-122.36, 37....",usp000jyuq,"{'time': 1359225564290, 'tz': None, 'type': 'e...",Feature,None,2.7,p000jyuq,http://earthquake.usgs.gov/fdsnws/event/1/quer...,NaN,13,...,",nc,us,",reviewed,1359225564290,M 2.6 - Northern California,0,earthquake,",dyfi,impact-text,origin,phase-data,",NaN,1422580916689,http://earthquake.usgs.gov/earthquakes/eventpa...
2,"{'type': 'Point', 'coordinates': [-122.2333, 3...",nc71928861,"{'time': 1359161957300, 'tz': -480, 'type': 'e...",Feature,None,3.3,71928861,http://earthquake.usgs.gov/fdsnws/event/1/quer...,0.008983,4,...,",nc,",REVIEWED,1359161957300,"M 1.8 - 3km ENE of Berkeley, California",0,earthquake,",dyfi,general-link,general-link,geoserve,nearb...",-480,1422653773801,http://earthquake.usgs.gov/earthquakes/eventpa...
3,"{'type': 'Point', 'coordinates': [-122.059, 38...",usp000jyts,"{'time': 1359135538700, 'tz': None, 'type': 'e...",Feature,None,3.4,p000jyts,http://earthquake.usgs.gov/fdsnws/event/1/quer...,NaN,369,...,",nc,us,",reviewed,1359135538700,M 2.9 - Northern California,0,earthquake,",cap,dyfi,impact-text,origin,phase-data,",NaN,1422439119296,http://earthquake.usgs.gov/earthquakes/eventpa...
4,"{'type': 'Point', 'coordinates': [-122.395, 38...",usp000jytk,"{'time': 1359106538230, 'tz': None, 'type': 'e...",Feature,None,2.4,p000jytk,http://earthquake.usgs.gov/fdsnws/event/1/quer...,NaN,3,...,",nc,us,",reviewed,1359106538230,M 2.0 - Northern California,0,earthquake,",dyfi,impact-text,origin,phase-data,",NaN,1422670638787,http://earthquake.usgs.gov/earthquakes/eventpa...
5,"{'type': 'Point', 'coordinates': [-119.6145, 3...",nc71069629,"{'time': 1359093554800, 'tz': -480, 'type': 'e...",Feature,None,NaN,71069629,http://earthquake.usgs.gov/fdsnws/event/1/quer...,0.152714,NaN,...,",nc,",REVIEWED,1359093554800,"M 0.0 - 33km SW of Smith Valley, Nevada",0,earthquake,",general-link,general-link,general-link,geoser...",-480,1360380540590,http://earthquake.usgs.gov/earthquakes/eventpa...
6,"{'type': 'Point', 'coordinates': [-119.6177, 3...",nc71928386,"{'time': 1359093513900, 'tz': -480, 'type': 'e...",Feature,None,NaN,71928386,http://earthquake.usgs.gov/fdsnws/event/1/quer...,0.152714,NaN,...,",nn,nc,",REVIEWED,1359093513900,"M 1.5 - 35km SW of Smith Valley, Nevada",0,earthquake,",cap,general-link,general-link,general-link,ge...",-480,1400530246000,http://earthquake.usgs.gov/earthquakes/eventpa...
7,"{'type': 'Point', 'coordinates': [-122.232, 37...",usp000jytd,"{'time': 1359090747530, 'tz': None, 'type': 'e...",Feature,None,2.0,p000jytd,http://earthquake.usgs.gov/fdsnws/event/1/quer...,NaN,7,...,",nc,us,",reviewed,1359090747530,"M 2.0 - San Francisco Bay area, California",0,earthquake,",cap,dyfi,impact-text,origin,phase-data,",NaN,1422611894841,http://earthquake.usgs.gov/earthquakes/eventpa...
8,"{'type': 'Point', 'coordinates': [-119.6242, 3...",nc71928341,"{'time': 1359087199600, 'tz': -480, 'type': 'e...",Feature,None,NaN,71928341,http://earthquake.usgs.gov/fdsnws/event/1/quer...,0.152714,NaN,...,",nc,",REVIEWED,1359087199600,"M 1.9 - 35km SW of Smith Valley, Nevada",0,earthquake,",general-link,general-link,general-link,geoser...",-480,1360380189213,http://earthquake.usgs.gov/earthquakes/eventpa...
9,"{'type': 'Point', 'coordinates': [-119.618, 38...",nc71928221,"{'time': 1359072385100, 'tz': -480, 'type': 'e...",Feature,None,NaN,71928221,http://earthquake.usgs.gov/fdsnws/event/1/quer...,0.152714,NaN,...,",nc,n

In [20]:
eventData["productList"] = eventData["types"].apply(lambda x: x.split(",")[1:-1])

In [21]:
productsToLoad = ["focal-mechanism", "moment-tensor"]

In [22]:
eventData["hasRequestedProducts"] = eventData["productList"].apply(lambda x: len(list(set(x) & set(productsToLoad)))>0)

In [23]:
eventsWithProductsToLoad = eventData[eventData["hasRequestedProducts"] == 1]

In [ ]:
session = requests.Session()
adapter = requests.adapters.HTTPAdapter(pool_connections=100, pool_maxsize=100)
session.mount('http://', adapter)
headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
# reqs = (grequests.post(SERVICE_URL, data=params[i % len(params)], headers=headers, session=session)
#             for i in xrange(no_requests))
detailUrlRequests = [grequests.get(url, hooks = {'response': processResponse }, session=session) 
                      for url in eventsWithProductsToLoad["detail"]]

detailData = {}

# for detailUrl in eventsWithProductsToLoad["detail"]:
for response in grequests.imap(detailUrlRequests, size=100):
#     response = requests.get(detailUrl)
    if response.status_code != 200:
        print("Error loading url {0}: {1}".format(response.url, response.text))
    else:
        print("Loading {0}".format(response.url))

    json_data = response.json()

    properties = json_data["properties"]
    products = properties["products"]

    eventid = "{0}{1}".format(properties["net"], properties["code"])

    for product in productsToLoad:

        if product in products:
            items = products[product]

            for item in items:
                detailDataFrame = pd.concat([DataFrame(item, index=[0]), DataFrame(item["properties"], index=[0])], axis=1)
                detailDataFrame["eventid"] = eventid

                if product not in detailData:
                    detailData[product] = detailDataFrame
                else:
                    detailData[product] = detailData[product].append(detailDataFrame)
#     time.sleep(5)
    

Loading http://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ci15277281&format=geojson
Loading http://earthquake.usgs.gov/fdsnws/event/1/query?eventid=nc71963935&format=geojson
Loading http://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ci15276993&format=geojson
Loading http://earthquake.usgs.gov/fdsnws/event/1/query?eventid=nc71963950&format=geojson
Loading http://earthquake.usgs.gov/fdsnws/event/1/query?eventid=nc71963000&format=geojson
Loading http://earthquake.usgs.gov/fdsnws/event/1/query?eventid=nc71958710&format=geojson
Loading http://earthquake.usgs.gov/fdsnws/event/1/query?eventid=nc71962650&format=geojson
Loading http://earthquake.usgs.gov/fdsnws/event/1/query?eventid=nc71955085&format=geojson
Loading http://earthquake.usgs.gov/fdsnws/event/1/query?eventid=nc72002511&format=geojson
Loading http://earthquake.usgs.gov/fdsnws/event/1/query?eventid=nc72001896&format=geojson
Loading http://earthquake.usgs.gov/fdsnws/event/1/query?eventid=nc71990576&format=geojson
Loading ht

In [27]:
detailData["moment-tensor"]

,beachball-source,beachball-type,code,contents,depth,derived-depth,derived-eventtime,derived-latitude,derived-longitude,derived-magnitude,...,t-axis-plunge,tensor-mpp,tensor-mrp,tensor-mrr,tensor-mrt,tensor-mtp,tensor-mtt,type,updateTime,version
0,BRK,Mwr,pde20130124232551220_7_M_BRK,NaN,NaN,8,NaN,NaN,NaN,NaN,...,5,1360000000000000,110000000000000,-330000000000000,370000000000000,220000000000000,-1030000000000000,moment-tensor,1415324993111,NaN
0,BRK,Mwr,pde20130124082148200_7_M_BRK,NaN,NaN,11,NaN,NaN,NaN,NaN,...,13,365000000000000,74000000000000,-2000000000000,102000000000000,87000000000000,-363000000000000,moment-tensor,1415324993008,NaN
0,BRK,Mwr,pde20130315030501610_9_M_BRK,NaN,NaN,14,NaN,NaN,NaN,NaN,...,12,54200000000000,7200000000000,-21000000000000,34400000000000,22100000000000,-33200000000000,moment-tensor,1415325020492,NaN
0,BRK,Mwr,pde20130309101309800_7_M_BRK,NaN,NaN,8,NaN,NaN,NaN,NaN,...,58,24400000000000,52800000000000,66100000000000,-59300000000000,10300000000000,-90500000000000,moment-tensor,1415325018852,NaN
0,NC,smi:nc.anss.org/momentTensor/TMTS,nc71988946_mt1,NaN,NaN,8,2013-05-10T05:12:24.820Z,37.3671989,-121.7332993,3.5,...,NaN,195700000000000,10680000000000,890000000000,52130000000000,88370000000000,-196600000000000,moment-tensor,1368166216680,NaN
0,BRK,Mwr,pde20130510051224900_8_M_BRK,NaN,NaN,8,NaN,NaN,NaN,NaN,...,6,196000000000000,11000000000000,1000000000000,52000000000000,88000000000000,-197000000000000,moment-tensor,1415325046000,NaN
0,NC,smi:nc.anss.org/momentTensor/TMTS,nc72016695_mt1,NaN,NaN,14,2013-06-28T03:52:39.480Z,36.7566986,-121.4701996,3.87,...,NaN,728800000000000,-143100000000000,30250000000000,-182800000000000,55220000000000,-759000000000000,moment-tensor,1373669263090,NaN
0,BRK,Mwr,pde20130628035239400_6_M_BRK,NaN,NaN,14,NaN,NaN,NaN,NaN,...,12,729000000000000,-143000000000000,30000000000000,-183000000000000,55000000000000,-759000000000000,moment-tensor,1415325067908,NaN
0,NC,smi:nc.anss.org/momentTensor/TMTS,nc72032105_mt1,NaN,NaN,8,2013-07-24T00:13:00.220Z,38.4482002,-122.735199,2.87,...,NaN,24520000000000,-3420000000000,-3650000000000,7680000000000,4780000000000,-20870000000000,moment-tensor,1374686601070,NaN
0,BRK,Mwr,pde20130724001300200_8_M_BRK,NaN,NaN,8,NaN,NaN,NaN,NaN,...,5,24500000000000,-3400000000000,-3700000000000,7700000000000,4800000000000,-20900000000000,moment-tensor,1415325079844,NaN


In [ ]:
# with open("detail.json") as json_file:
#     json_data = json.load(json_file)

# detailData = {}
# properties = json_data["properties"]
# products = properties["products"]

# eventid = "{0}{1}".format(properties["net"], properties["code"])

# for product in productsToLoad:

#     if product in products:
#         items = products[product]

#         for item in items:
#             detailDataFrame = pd.concat([DataFrame(item, index=[0]), DataFrame(item["properties"], index=[0])], axis=1)
#             detailDataFrame["eventid"] = eventid

#             if product not in detailData:
#                 detailData[product] = detailDataFrame
#             else:
#                 detailData[product] = detailData[product].append(detailDataFrame)

# detailData["moment-tensor"]

In [28]:
detailData["focal-mechanism"]

,beachball-source,code,contents,depth,evaluation-status,eventParametersPublicID,eventid,eventsource,eventsourcecode,eventtime,...,properties,quakeml-publicid,review-status,source,status,t-axis-azimuth,t-axis-plunge,type,updateTime,version
0,NC,nc71963935_fm1,NaN,NaN,NaN,quakeml:nc.anss.org/catalog/NC/1/136512530024,nc71963935,nc,71963935,2013-03-31T17:27:15.220Z,...,NaN,NaN,automatic,nc,UPDATE,NaN,NaN,focal-mechanism,1365125300240,NaN
0,NC,nc71963950_fm2,NaN,NaN,NaN,quakeml:nc.anss.org/catalog/NC/1/136520797095,nc71963950,nc,71963950,2013-03-31T18:27:17.190Z,...,NaN,NaN,automatic,nc,UPDATE,NaN,NaN,focal-mechanism,1365207970950,NaN
0,NC,nc71963950_fm1,NaN,NaN,NaN,quakeml:nc.anss.org/catalog/NC/1/136520796775,nc71963950,nc,71963950,2013-03-31T18:27:17.190Z,...,NaN,NaN,automatic,nc,UPDATE,NaN,NaN,focal-mechanism,1365207967750,NaN
0,NC,nc71963000_fm1,NaN,NaN,NaN,quakeml:nc.anss.org/catalog/NC/1/136512750685,nc71963000,nc,71963000,2013-03-30T04:19:29.140Z,...,NaN,NaN,automatic,nc,UPDATE,NaN,NaN,focal-mechanism,1365127506850,NaN
0,NC,71958710_fm2,NaN,NaN,NaN,quakeml:nc.anss.org/catalog/NC/1/136503176708,nc71958710,nc,71958710,2013-03-23T02:49:28.390Z,...,NaN,NaN,automatic,nc,UPDATE,NaN,NaN,focal-mechanism,1365031767080,NaN
0,NC,71958710_fm1,NaN,NaN,NaN,quakeml:nc.anss.org/catalog/NC/1/136503176408,nc71958710,nc,71958710,2013-03-23T02:49:28.390Z,...,NaN,NaN,automatic,nc,UPDATE,NaN,NaN,focal-mechanism,1365031764080,NaN
0,NC,nc71962650_fm1,NaN,NaN,NaN,quakeml:nc.anss.org/catalog/NC/1/136511311771,nc71962650,nc,71962650,2013-03-29T15:44:59.050Z,...,NaN,NaN,automatic,nc,UPDATE,NaN,NaN,focal-mechanism,1365113117710,NaN
0,NC,nc72002511_fm2,NaN,NaN,NaN,quakeml:nc.anss.org/catalog/NC/1/136995015279,nc72002511,nc,72002511,2013-05-29T21:13:36.460Z,...,NaN,NaN,automatic,nc,UPDATE,NaN,NaN,focal-mechanism,1369950152790,NaN
0,NC,nc72002511_fm1,NaN,NaN,NaN,quakeml:nc.anss.org/catalog/NC/1/136995014936,nc72002511,nc,72002511,2013-05-29T21:13:36.460Z,...,NaN,NaN,automatic,nc,UPDATE,NaN,NaN,focal-mechanism,1369950149360,NaN
0,NC,nc72001896_fm1,NaN,NaN,NaN,quakeml:nc.anss.org/catalog/NC/1/137115855854,nc72001896,nc,72001896,2013-05-28T21:27:37.420Z,...,NaN,NaN,automatic,nc,UPDATE,NaN,NaN,focal-mechanism,1371158558540,NaN
